# Scratch

---

### Imports & Setup

Installs

In [11]:
# Run this cell to install required packages in your Jupyter notebook environment
%pip install openai sqlalchemy sqlmodel psycopg2-binary pandas

Note: you may need to restart the kernel to use updated packages.


General imports

In [2]:
from datetime import datetime, timedelta
from typing import List, Optional

Connect to DB

In [7]:
from sqlalchemy import create_engine
from sqlmodel import Session, SQLModel, select

# PostgreSQL connection URL
DATABASE_HOST = "localhost"  # Service name in docker-compose.yml
DATABASE_PORT = "5432"
DATABASE_NAME = "updatesdb"
DATABASE_USER = "postgres"
DATABASE_PASSWORD = "your_password"

ENGINE_URL = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"

engine = create_engine(ENGINE_URL, echo=True, pool_pre_ping=True)

# Optional: If you need to create the tables in your database (uncomment if necessary)
# SQLModel.metadata.create_all(engine)

Models

In [ ]:
from sqlmodel import SQLModel, Field, Relationship
from sqlalchemy import Column, BigInteger

class ChatType(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    type_name: str = Field(sa_column_kwargs={"unique": False})

class Chat(SQLModel, table=True):
    id: int = Field(primary_key=True)
    chat_id: Optional[int] = Field(default=None, sa_column=Column(BigInteger()))
    all_members_are_administrators: bool = Field()
    title: str = Field()
    type_id: int = Field(foreign_key="chattype.id")
    type: ChatType = Relationship()

class User(SQLModel, table=True):
    id: int = Field(primary_key=True, index=True)
    user_id: Optional[int] = Field(default=None)  # Telegram user ID
    first_name: str = Field()
    last_name: Optional[str] = Field(default=None)
    is_bot: bool = Field()
    language_code: str = Field()
    username: Optional[str] = Field()

class Message(SQLModel, table=True):
    id: int = Field(primary_key=True)
    message_id: int = Field(default=None)
    channel_chat_created: bool = Field()
    chat_id: int = Field(foreign_key="chat.id")
    chat: Chat = Relationship()
    date: int = Field()
    delete_chat_photo: bool = Field()
    from_user_id: int = Field(foreign_key="user.id")
    from_user: User = Relationship()
    group_chat_created: bool = Field()
    reply_to_message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    reply_to_message: 'Message' = Relationship(sa_relationship_kwargs={"remote_side": "Message.id"})
    supergroup_chat_created: bool = Field()
    text: Optional[str] = Field()

class Update(SQLModel, table=True):
    update_id: int = Field(primary_key=True)
    message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    message: Optional[Message] = Relationship()

---

### Display Tables (in pandas)

In [9]:
import pandas as pd

def display_table_data_pandas(model_class):
    with Session(engine) as session:
        statement = select(model_class)
        results = session.exec(statement).all()
        return pd.DataFrame([result.__dict__ for result in results])

In [10]:
# Display data using pandas for better formatting
data = display_table_data_pandas(Message)
data.head()

2024-05-11 12:14:02,717 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-11 12:14:02,717 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:14:02,718 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-11 12:14:02,718 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:14:02,719 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-11 12:14:02,719 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:14:02,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-11 12:14:02,726 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text 
FROM message
2024-05-11 12:14:02,726 INFO sqlalchemy.engine.Engine [generated in 0.00041s] {}
2024-05-11 12:14:02,728 INFO sqlalchemy.engine.Engine ROLLBACK


,_sa_instance_state,id,chat_id,delete_chat_photo,group_chat_created,supergroup_chat_created,message_id,channel_chat_created,date,from_user_id,reply_to_message_id,text
0,<sqlalchemy.orm.state.InstanceState object at ...,1,2,False,False,False,22497,False,1715268507,2,NaN,are you saying this due to the whole wallet wh...
1,<sqlalchemy.orm.state.InstanceState object at ...,2,1,False,False,False,22501,False,1715270079,1,1.0,No. the same whitelisting issues exists with 7...
2,<sqlalchemy.orm.state.InstanceState object at ...,3,3,False,False,False,248,False,1715270331,3,NaN,"Ok Archiving is working fine for now, will sta..."
3,<sqlalchemy.orm.state.InstanceState object at ...,4,4,False,False,False,22502,False,1715270488,4,NaN,"From the POV of wallets, I think the whitelist..."
4,<sqlalchemy.orm.state.InstanceState object at ...,5,6,False,False,False,22502,False,1715270488,6,NaN,"From the POV of wallets, I think the whitelist..."


---

### Reconstruct Chat History

This function takes a list of conversations and message and returns the index of the conversation the message belongs to, or none

In [95]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

def select_conversation_for_message(conversations: List[List[Message]], message: Message) -> int:
    # Instantiate OpenAI chat client
    gpt = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # Prompt template
    prompt = """
This is a list of indexed conversations:

{conversations}

This is a message that needs to be placed in one of the conversations above:

{message}

Please reply with the index of the conversation that the message belongs to.
Think about the context of the conversation and how the message fits in.
Your response will be converted into an int for further processing, so please 
reply with only the number and absolutely nothing else.
"""

    conversation_text = ""

    for index, conversation in enumerate(conversations):
        # Generate the conversation text with an index
        conversation_text = conversation_text + f"\n{index}: \n" + "\n".join([f"{message.from_user.username}: {message.text}" for message in conversation])

    prompt = prompt.format(conversations=conversation_text, message=f"{message.from_user.username}: {message.text}")

    # Generate the completion
    completion = gpt.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=10,
        temperature=0.1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    # Extract the completion choice
    choice = int(completion.choices[0].message.content.strip())

    # # Return the choice if it is valid
    if 0 <= choice < len(conversations):
        return choice
        
    return None

In [96]:
# Dummy conversations for testing
# Creating dummy users
user1 = User(id=1, username="Alice")
user2 = User(id=2, username="Bob")

# Creating dummy messages
message1 = Message(id=1, from_user=user1, text="Hello, how are you?", reply_to_message_id=None)
message2 = Message(id=2, from_user=user2, text="I'm good, thanks! And you?", reply_to_message_id=1)
message3 = Message(id=3, from_user=user1, text="Great to hear that, I'm fine too.", reply_to_message_id=2)

message4 = Message(id=4, from_user=user2, text="Have you seen the latest movie?", reply_to_message_id=None)
message5 = Message(id=5, from_user=user1, text="Not yet, is it good?", reply_to_message_id=4)

# Creating test message
# message = Message(id=6, from_user=user2, text="It was an amazing movie!", reply_to_message_id=None)
message = Message(id=6, from_user=user2, text="Ok, I'll watch it this week then.", reply_to_message_id=None)

# Organizing messages into conversations
conversations = [
    [message1, message2, message3],  # Conversation 1
    [message4, message5]             # Conversation 2
]

# Test the function
select_conversation_for_message(conversations, message)

1


This function takes a list of conversations and a message and appends the message to the conversation it belongs to. 

In [ ]:
def select_conversation_for_message_and_add(conversations: List[List[Message]], message: Message, openai_client):
    # Check if the message is a direct reply
    if hasattr(message, 'reply_to_message_id') and message.reply_to_message_id:
        for conversation in conversations:
            for msg in conversation:
                if msg.id == message.reply_to_message_id:
                    conversation.append(message)
                    return
    
    # Use AI to find the right conversation for non-reply messages
    if len(conversations) > 0:
        conversation_index = select_conversation_for_message(conversations, message, openai_client)
        if conversation_index is not None:
            conversations[conversation_index].append(message)
            return

    # If no conversation fits or list is empty, start a new conversation
    conversations.append([message])

In [10]:
def reconstruct_chat_as_conversations(engine, start_time: datetime, end_time: datetime) -> List[List[Message]]:
    conversations = []  # This will store lists of messages categorized into conversations
    with Session(engine) as session:
        # Fetch all messages within the given time range, ordering by the date
        statement = select(Message).where(
            Message.date >= start_time.timestamp(),  # Assuming 'date' is stored as a UNIX timestamp
            Message.date <= end_time.timestamp()
        ).order_by(Message.date)
        results = session.exec(statement).all()

        for message in results:
            select_conversation_for_message_and_add(conversations, message)
    
    return conversations

In [7]:
# Get dates
today = datetime.now()
previous_day = today - timedelta(days=1)
# Set start_time to the beginning of the previous day (00:00:00)
start_time = datetime(previous_day.year, previous_day.month, previous_day.day, 0, 0, 0)
# Set end_time to the end of the previous day (23:59:59)
end_time = datetime(previous_day.year, previous_day.month, previous_day.day, 23, 59, 59)

# Get chat history text
chat_text = reconstruct_chat_as_text(engine, start_time, end_time)

2024-05-11 09:12:25,250 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-11 09:12:25,251 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text, "user".id AS id_1, "user".user_id, "user".first_name, "user".last_name, "user".is_bot, "user".language_code, "user".username 
FROM message JOIN "user" ON "user".id = message.from_user_id 
WHERE message.date >= %(date_1)s AND message.date <= %(date_2)s ORDER BY message.date
2024-05-11 09:12:25,252 INFO sqlalchemy.engine.Engine [generated in 0.00025s] {'date_1': 1715299200.0, 'date_2': 1715385599.0}
2024-05-11 09:12:25,258 INFO sqlalchemy.engine.Engine SELECT message.id AS message_id_1, message.message_id AS message_message_id, message.channel_chat_created AS message_channel_chat_created, message.chat_id AS message_c

In [8]:
print(chat_text)

big_tech_sux: <replying to: vb271828: The main value of code immutability is so that other contracts can point to specific addresses on chain and use them as libraries. And in ERC4337, accessing mutable libraries becomes a DoS attack vector> yea, maybe i'm a bit late to the discussion but it seems like a bit of a footgun if contracts can't determine if a given code is mutable (i.e. it's an eoa) or not
big_tech_sux: but to my small mind, ephemeral code as defined in 7702 should behave basically like initializing a selfdestructing contract, and then immediately delegatecalling into it. and maybe we get an IS_EOA opcode??
big_tech_sux: but to my small brain, ephemeral code as defined in 7702 should behave basically like initializing a selfdestructing contract, and then immediately delegatecalling into it. and maybe we get an IS_EOA opcode??
big_tech_sux: but to my small brain, ephemeral code as defined in 7702 should behave basically like initializing a selfdestructing contract, and then 

---

### Extract topics

In [42]:
from openai import OpenAI

client = OpenAI(api_key="your_api_key_here")

def extract_topics_from_chat(chat_text):
    # Using the chat-specific completion API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Use an appropriate model, like 'gpt-3.5-turbo'
        messages=[{"role": "system", "content": "Extract the main topics from the following group chat:"},
                  {"role": "user", "content": chat_text}],
        max_tokens=300,
        temperature=0.5,
        stop=["\n\n"]
    )
    
    return response.choices[0].message.content

In [43]:
# Assume the OpenAI client is initialized and `extract_topics_from_chat` is defined
dummy_chat_text = """
Alice: I just got a new kitten yesterday!
Bob: Oh, that's awesome! What breed?
Alice: She's a Maine Coon, absolutely adorable.
Charlie: Speaking of pets, I've been thinking about getting a fish tank.
Alice: Fish are cool, but they seem like a lot of work.
Bob: <replying to: Charlie: Speaking of pets, I've been thinking about getting a fish tank.> It's not too bad, you should definitely go for it. I had one as a kid.
Charlie: Any recommendations for beginner fish?
Bob: Goldfish are pretty straightforward, but bettas have more personality.
Alice: <replying to: Bob: Oh, that's awesome! What breed?> Cats are easier to handle though, don't you think?
Bob: <replying to: Alice: Cats are easier to handle though, don't you think?> Definitely, cats are more independent.
Charlie: I like dogs more though. They're more interactive.
Alice: I'm allergic to dogs unfortunately.
Bob: <replying to: Charlie: I like dogs more though. They're more interactive.> Dogs are great for active people. Always ready to go out.
Charlie: <replying to: Bob: Goldfish are pretty straightforward, but bettas have more personality.> I might consider a betta then.
Alice: Just make sure the tank is big enough, they need space to thrive.
Bob: <replying to: Alice: Just make sure the tank is big enough, they need space to thrive.> Yeah, and keep the water clean!
"""

topics = extract_topics_from_chat(dummy_chat_text)

In [44]:
print(topics)

1. Pets (kitten, fish, cats, dogs)
2. Fish tanks and fish care (beginner fish, tank size, water cleanliness)
3. Different pet preferences (cats, dogs, fish)
4. Allergies to pets
5. Pet ownership responsibilities and considerations
